In [34]:
import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [35]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [36]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [37]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/maxtmax/maxtmax_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.maxtmax

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [38]:

# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

#~

In [39]:

# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [40]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [41]:
region_all.to_dataframe(name='value').reset_index().to_csv('maxtmax2049-2079.csv')

In [42]:
#new variable- precipitation projections 



import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [43]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [44]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [45]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/pr/pr_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.pr

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [46]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [47]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [48]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [49]:
region_all.to_dataframe(name='value').reset_index().to_csv('pr_projection_2049-2079.csv')

In [50]:
#new data mean daily temperature 


import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [51]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [52]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [53]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/tas/tas_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.tas

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [54]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [55]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [56]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [57]:
region_all.to_dataframe(name='value').reset_index().to_csv('tas_projection_2049-2079.csv')

In [58]:
#new variable: number of dangerously hot days 


import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [59]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [60]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [61]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/ndhi/ndhi_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.ndhi

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [62]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [63]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [64]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [65]:
region_all.to_dataframe(name='value').reset_index().to_csv('ndhi_projection_2049-2079.csv')

In [70]:
#new variable -- drought duration based on 6 month evapotranspiration index

import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [71]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [72]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [73]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/spei6dd/spei6dd_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.spei6dd

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [74]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [75]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [76]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [77]:
region_all.to_dataframe(name='value').reset_index().to_csv('drought_duration_projection_2049-2079.csv')

In [1]:
#new variable--- max five day precipitation 
import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [2]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [3]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

# load the raw data
fnm = "/data/met/plot-explorer/data/rx5day/rx5day_12km_ens-mean-maps_20491201-20791130_annual.nc"
ds = xr.open_dataset(fnm)
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
da = ds.rx5day

rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [6]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [7]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [8]:
region_all.to_dataframe(name='value').reset_index().to_csv('max_5_day_precipitation_projection_2049-2079.csv')